In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (8, 4)

df = pd.read_csv('data/raw/titanic.csv')
df.head()

# Preprocesado y Modelos

#En este notebook realizamos el preprocesamiento del dataset del Titanic y probamos diferentes modelos de Machine Learning para predecir la supervivencia. Este paso forma parte del pipeline del proyecto.



# Preprocesado y modelos – Titanic

En este notebook probamos distintos **pasos de preprocesado** y **modelos de Machine Learning** 
para predecir la supervivencia de los pasajeros del Titanic.

Partimos del análisis exploratorio realizado en el notebook `EDA de Isabella` y aquí 
nos centramos en:

1. Seleccionar las variables más relevantes.
2. Definir un pipeline de preprocesado (nulos, codificación, escalado).
3. Entrenar y comparar varios modelos de clasificación.
4. Elegir un modelo base para el servicio de predicción.


## 1. Carga de datos

Cargamos el mismo dataset utilizado en el EDA, a partir de la carpeta `data/raw`.


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


df = pd.read_csv("data/raw/titanic.csv")
df.head()


## 2. Selección de variables

#Para este experimento vamos a usar las siguientes columnas como *features*:

##- `pclass` – clase del billete  
#- `sex` – sexo  
#- `age` – edad  
#- `sibsp` – nº de hermanos/cónyuges a bordo  
#- `parch` – nº de padres/hijos a bordo  
#- `fare` – tarifa pagada  
#- `embarked` – puerto de embarque  

#La variable objetivo será `survived` (0 = no sobrevive, 1 = sobrevive).


num_features = ["Age", "SibSp", "Parch", "Fare"]
cat_features = ["Sex", "Embarked"]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features)
    ]
)

X = df.drop("Survived", axis=1)
y = df["Survived"]


In [ ]:
feature_cols = ["pclass", "sex", "age", "sibsp", "parch", "fare", "embarked"]
target_col = "survived"

X = df[feature_cols]
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape


## 3. Pipeline de preprocesado

Definimos un pipeline que haga automáticamente:

- Imputación de valores nulos en variables numéricas (con la media).
- Imputación de nulos en categóricas (con la categoría más frecuente).
- Escalado de variables numéricas.
- Codificación *one-hot* de variables categóricas.


In [ ]:
from sklearn.impute import SimpleImputer

numeric_features = ["age", "sibsp", "parch", "fare"]
categorical_features = ["pclass", "sex", "embarked"]

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)


## 4. Modelos a comparar

Vamos a probar dos modelos sencillos:

1. **Regresión logística** – modelo lineal rápido y fácil de interpretar.  
2. **Random Forest** – conjunto de árboles de decisión, capaz de capturar relaciones no lineales.

En ambos casos utilizaremos el *mismo pipeline de preprocesado* definido antes.


In [ ]:
# Pipeline con Regresión Logística
log_reg_clf = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression(max_iter=1000)),
    ]
)

# Pipeline con Random Forest
rf_clf = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", RandomForestClassifier(
            n_estimators=200,
            random_state=42
        )),
    ]
)

models = {
    "Logistic Regression": log_reg_clf,
    "Random Forest": rf_clf,
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"Modelo: {name}")
    print(f"Accuracy en test: {acc:.3f}")
    print(classification_report(y_test, y_pred))
    print("-" * 60)


## 5. Comparación de resultados

En la celda anterior hemos entrenado ambos modelos y calculado el 
**accuracy** sobre el conjunto de test. A continuación mostramos 
un pequeño resumen comparativo.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model_lr = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression())
])

model_lr.fit(X, y)
y_pred_lr = model_lr.predict(X)
accuracy_lr = accuracy_score(y, y_pred_lr)
accuracy_lr


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier())
])

model_rf.fit(X, y)
y_pred_rf = model_rf.predict(X)
accuracy_rf = accuracy_score(y, y_pred_rf)
accuracy_rf


## 6. Conclusiones

- El modelo con mejor rendimiento ha sido: **(rellenar según resultados)**.
- El pipeline de preprocesado nos permite manejar nulos y variables 
  categóricas de forma automática.
- Este notebook sirve como base para decidir qué modelo integrar en el 
  servicio de predicción del proyecto (`src/`).

> **Nota:** aquí puedes escribir 3–4 bullets con tus conclusiones personales 
> (lo que pide el profe).
